# Для данного датасета обучите дерево решений (для обучения используйте массивы с ключами x, y),
затем предскажите классы для данных доступных в файле по ключу x_test.

В качестве ответа запишите метку класса - 1 или 0

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

Загружаем данные и формируем датасеты

In [87]:
data = np.load('content/train_data.npz')
X = pd.DataFrame(data['x'])
y = pd.Series(data['y'])
X_test = pd.DataFrame(data['x_test'])

Масштабируем данные, для алгоритмов решающего дерева и прочих "деревянных алгоритмов" данная операция не нужна, однако, никогда не вредно повторить.

In [65]:
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X))
X_test_scaler = pd.DataFrame(sc.transform(X_test))

Деревянные алгоритмы легко переобучаются, поэтому необходим подбор гипперпараметров. В нашем случае по сете подбираем максимальную глубину, критерий и прочее..

In [52]:
params = {'max_depth': np.arange(1, 20, 1),
          'max_features': [None, 'sqrt', 'log2'],
          'criterion': ["gini", "entropy"]}
gs = GridSearchCV(DecisionTreeClassifier(), params, cv=3, scoring='roc_auc', n_jobs=2)
gs.fit(X_train, y)
gs.best_score_, gs.best_estimator_

(0.9600779967159278, DecisionTreeClassifier(max_depth=3, max_features='log2'))

Используя модель с наилучшими гипперпараметрами, снова обучаем тренировочную выборку, делаем предсказание на тестовой и выводим результат.

In [53]:
model = gs.best_estimator_
model.fit(X_train, y)
pred_proba = pd.DataFrame(model.predict_proba(X_test_scaler), columns=['prob 0', 'prob 1'])
pred = model.predict(X_test_scaler)
pred_proba['class'] = pred
pred_proba

,prob 0,prob 1,class
0,0.015625,0.984375,1
1,0.015625,0.984375,1
2,0.993711,0.006289,0
3,0.777778,0.222222,0
4,0.015625,0.984375,1
5,0.993711,0.006289,0
6,0.015625,0.984375,1
7,0.015625,0.984375,1
8,0.015625,0.984375,1
9,0.015625,0.984375,1


В проверочной системе Stepik ОШИБКА! Имейте это ввиду. Мы решили правильно!

# Одно из возможных решений!

In [82]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

data = np.load('content/train_data.npz')

x = pd.DataFrame(data["x"])
y = pd.DataFrame(data["y"])

x_test = pd.DataFrame(data["x_test"])

correlation = 0.9
df = x[x.corr() > correlation].fillna(0)
df_test = x_test[x_test.corr() > correlation].fillna(0)

classifier = DecisionTreeClassifier()
classifier.fit(df, y)

y_pred = classifier.predict(df_test)

print(y_pred)

[1 1 0 1 1 1 0 1 0 1]


In [84]:
from sklearn.ensemble import GradientBoostingClassifier

In [93]:
params = {'n_estimators': np.arange(10, 200, 20),
          'max_depth': np.arange(2, 20, 3)
          }
gs = GridSearchCV(GradientBoostingClassifier(), params, cv=3, scoring='roc_auc', verbose=2)
gs.fit(X, y)
gs.best_score_, gs.best_estimator_

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] END .......................max_depth=2, n_estimators=10; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=10; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=10; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=30; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=30; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=30; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=50; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=50; total time=   0.1s
[CV] END .......................max_depth=2, n_estimators=50; total time=   0.0s
[CV] END .......................max_depth=2, n_estimators=70; total time=   0.1s
[CV] END .......................max_depth=2, n_estimators=70; total time=   0.0s
[CV] END .......................max_depth=2, n_

(0.9921182266009853, GradientBoostingClassifier(max_depth=2, n_estimators=150))

In [101]:
model = GradientBoostingClassifier(n_estimators=1500, max_depth=2, learning_rate=0.01)
model.fit(X, y)
pred = model.predict(X_test)
print(pred)

[1 1 0 1 1 0 1 1 1 1]
